In [3]:
from settings import RABBIT_ACCOUNT, RABBIT_PASSWORD, RABBIT_HOST, RABBIT_PORT, EVENT_QUEUE

In [7]:
import pandas as pd

In [25]:
from datetime import datetime

In [4]:
import pika
import json
class Queue():
    def __init__(self, host, port, username, password):
        self.connection = pika.BlockingConnection(pika.ConnectionParameters(host,port=port,credentials=pika.PlainCredentials(username=username,password=password), heartbeat=0))
        self.channel = self.connection.channel()
        self.channel.basic_qos(prefetch_count=1)
    def DeclareQueue(self, queuename):
        self.channel.queue_declare(queue = queuename)
    def AddToQueue(self, queuename, data):
        self.channel.basic_publish(exchange = '', routing_key = queuename, body = json.dumps(data), properties = pika.BasicProperties(delivery_mode=2))
    def GetFromQueue(self, queuename, callback, auto_ack):
        self.channel.basic_consume(queue = queuename, on_message_callback = callback,auto_ack= auto_ack)
        self.channel.start_consuming()
    def Close(self):
        self.connection.close()

In [5]:
event_queue = Queue(RABBIT_HOST, RABBIT_PORT, RABBIT_ACCOUNT, RABBIT_PASSWORD)

In [6]:
EVENT_QUEUE

'EventSourceURL'

In [8]:
event_df = pd.read_csv('eventsourcepage.csv')

In [10]:
event_df.head()

,EID,EventSourcePage,label
0,447,http://ewda.tw/,1
1,457,http://bambooculture.com/taxonomy/term/3411,1
2,460,http://investmentcommunity16888.blogspot.com/s...,1
3,463,http://www.chailease.org.tw/ugC_Activity.asp,1
4,464,https://www.ncu.edu.tw/events,1


In [21]:
#Check if there is duplicated url
sum(event_df['EventSourcePage'].duplicated())

0

In [22]:
import validators

In [24]:
print("Adding urls to Event_queue...")
for url in event_df['EventSourcePage'].values.tolist():
    if validators.url(url):
#         print(url)
        event_queue.AddToQueue(EVENT_QUEUE, url)
print("Finish adding url to Event_queue...")

Adding urls to Event_queue...
Finish adding url to Event_queue...


In [26]:
## Current Time
datetime.now()

datetime.datetime(2021, 4, 20, 17, 43, 41, 130132)